# Twitch data analysis

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
from pandas import json_normalize
from collections import Counter

In [2]:
completelist = [os.path.join(path, name) for path, subdirs, files in os.walk("./files_stream/") for name in files]
print("Number of files:", len(completelist))
print(*completelist[:10], sep = "\n")

Number of files: 10
./files_stream/2022-04-20_16-41.json
./files_stream/2022-04-20_16-42.json
./files_stream/2022-04-20_16-43.json
./files_stream/2022-04-20_16-44.json
./files_stream/2022-04-20_16-45.json
./files_stream/2022-04-20_16-46.json
./files_stream/2022-04-20_16-47.json
./files_stream/2022-04-20_16-48.json
./files_stream/2022-04-20_16-49.json
./files_stream/2022-04-20_16-50.json


### Sampling frequency

In [3]:
#set up sample frequency: 1=5min, 2=10min, 3=15min ecc.
freq= 2
flist= completelist[::freq]

In [4]:
for i in range(0, len(flist)):
    print(flist[i])

./files_stream/2022-04-20_16-41.json
./files_stream/2022-04-20_16-43.json
./files_stream/2022-04-20_16-45.json
./files_stream/2022-04-20_16-47.json
./files_stream/2022-04-20_16-49.json


### Import selected files

In [5]:
data_dict = {}

for file in flist:
    
    #extract timestamp
    start = './files_stream/'
    end = '.json'
    time = file[len(start): - len(end)]
    time_obj = datetime.strptime(time, '%Y-%m-%d_%H-%M')
    
    with open(file, 'r') as f:
        data_dict[time_obj] = json.load(f)

print("Number of keys (time intervals):", len(data_dict.keys()))

Number of keys (time intervals): 5


Verifichiamo la differenza tra viewer e spect

In [6]:
#aggiungere ciclo for su tutti gli istanti

for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    
    print(f"{streamer}\n\
            Viewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
            Spect Count: {len(data_dict[time_obj][streamer]['spect'])}\
            Difference: {data_dict[time_obj][streamer]['diff_count']}")

pizfn
            Viewer Count: 4290            Spect Count: 3058            Difference: 1232
Cerbero_Podcast
            Viewer Count: 3862            Spect Count: 3039            Difference: 823
Xiuder_
            Viewer Count: 2876            Spect Count: 2019            Difference: 857
GameStopIT
            Viewer Count: 2183            Spect Count: 799            Difference: 1384
Paoloidolo
            Viewer Count: 2142            Spect Count: 1573            Difference: 569
ilMasseo
            Viewer Count: 1801            Spect Count: 1398            Difference: 403
savyultras90
            Viewer Count: 1273            Spect Count: 938            Difference: 335
voghelita
            Viewer Count: 1251            Spect Count: 990            Difference: 261
Yoshi93
            Viewer Count: 1239            Spect Count: 946            Difference: 293
gskianto
            Viewer Count: 1098            Spect Count: 896            Difference: 202
Dada
            Viewer Count: 9

            Viewer Count: 1            Spect Count: 6            Difference: -5
sonnyttv__
            Viewer Count: 1            Spect Count: 6            Difference: -5
PixelWorldItalia
            Viewer Count: 1            Spect Count: 8            Difference: -7
camtele3tv
            Viewer Count: 1            Spect Count: 32            Difference: -31
BlazeOmicron99
            Viewer Count: 1            Spect Count: 9            Difference: -8
valentinank
            Viewer Count: 1            Spect Count: 4            Difference: -3
giuseppe4989
            Viewer Count: 1            Spect Count: 9            Difference: -8
GamerInsideIT
            Viewer Count: 1            Spect Count: 2            Difference: -1
icrxnusss
            Viewer Count: 1            Spect Count: 7            Difference: -6
fox_caliburn
            Viewer Count: 1            Spect Count: 5            Difference: -4
MattexUnstable
            Viewer Count: 1            Spect Count: 5            Di

Le differenze sono dovute a:

- bot rimossi dal viewer count
- persone non connesse in chat
- aggiornamento dati diverso tra i due

### Remove bot

In [7]:
bot_list= pd.read_csv('bot_list.csv')
bot_list= bot_list['Twitch Username'].tolist()
bot_list[:10]

['turbopascai',
 'tteyyd',
 'twitchdetails',
 '0liviajadeee',
 'judgejudysiayer',
 'thelurkertv',
 'nothingbutgay',
 'streamers_on_discord',
 '0_applebadapple_0',
 'agressive_sock']

In [8]:
def remove_bot(a):
    clean_list = []
    for chatter in a:
        if chatter not in bot_list:
            clean_list.append(chatter)
    return clean_list

In [9]:
def remove_bot2(spect_with_bot):
    return list(set(spect_with_bot) - set(bot_list))

In [10]:
for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect']= remove_bot(data_dict[time_obj][streamer]['spect'])

In [11]:
for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect']= remove_bot2(data_dict[time_obj][streamer]['spect'])

### Obtain streamer list and count

In [24]:
streamer = set()

for istante in data_dict.keys():
    new = set(data_dict[istante].keys()) - streamer
    notnew = set(data_dict[istante].keys()) - new
    
    #if
    streamer
    
    #streamer_repeated.extend(list(data_dict[istante].keys()))

#len(streamer_repeated)
type(notnew)

set

In [12]:
streamer_repeated = []

for istante in data_dict.keys():
    streamer_repeated.extend(list(data_dict[istante].keys()))

len(streamer_repeated)

10735

In [13]:
streamer_dict = dict(Counter(streamer_repeated))
len(streamer_dict.keys())

2147

#### Cut streamer list by count

In [15]:
min_stream_time = 2
print(f'Consideriamo solo chi ha streammato per almeno {5 * freq * min_stream_time} minuti')

streamer_dict_cut = {key:val for key, val in streamer_dict.items() if val > min_stream_time}
streamer_list_cut = [*streamer_dict_cut]
len(streamer_dict_cut.keys())

Consideriamo solo chi ha streammato per almeno 20 minuti


2147

### Obtain spect list and count